In [1]:
%matplotlib inline
import pandas as pd

from recommend import recommend_with_cosine_similarity

In [2]:
from IPython.core.display import HTML
css = open('table.css').read() + open('notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [3]:
video_games = pd.read_json("videogames.json")
video_games.head(5)

,brand,categories,description,imUrl,price,productID,related,salesRank,title
1012282,Dark Age Miniatures,"[[Video Games, PC, Games]]",None,http://ecx.images-amazon.com/images/I/61IaJuP1...,31.23,0984529527,None,{u'Toys & Games': 751946},Dark Age Apocalypse: Forcelists HC
1315355,None,"[[Video Games, PC, Games]]",Microsoft's Flight Simulator X is now making i...,http://ecx.images-amazon.com/images/I/41TaIYT%...,27.64,1557556741,"{u'also_bought': [u'B002JZH9E4'], u'buy_after_...",{u'Video Games': 61369},None
1321069,None,"[[Video Games, PC, Games]]",CYBERPUNK VAMPIRE ROLE-PLAYING PC GAME SET IN ...,http://ecx.images-amazon.com/images/I/41HClAKt...,NaN,1558843477,None,{u'Video Games': 56174},None
1336988,None,"[[Video Games, PC, Games]]",None,http://ecx.images-amazon.com/images/I/41PBCXQ3...,43.99,1563820412,None,{u'Software': 47380},None
1377319,Palladium Books,"[[Video Games, Nintendo DS, Games]]",The Rifter is your doorway to unlimited imagin...,http://ecx.images-amazon.com/images/I/41r7cYfc...,9.55,1574571931,"{u'also_bought': [u'157457194X', u'1574571923'...",{u'Toys & Games': 594770},Rifts RPG: The Rifter&reg; #56


In [4]:
ratings = pd.read_csv("ratings.csv")
ratings.head(5)

,userID,productID,ratings,timestamp
0,A24SSUT5CSW8BH,0078764343,5.0,1377302400
1,AK3V0HEBJMQ7J,0078764343,4.0,1372896000
2,A10BECPH7W8HM7,043933702X,5.0,1404950400
3,A2PRV9OULX1TWP,043933702X,5.0,1386115200
4,AE7GUHCDQQ4UI,043933702X,1.0,1366156800


#### Filter out video games which have no title

In [5]:
video_games = video_games[~video_games.title.isnull()]

#### Filter out video games which have less than 5 ratings

In [6]:
productID_counts = ratings['productID'].value_counts()
ratings = ratings[ratings['productID'].isin(productID_counts[productID_counts >= 5].index)]

#### Filter out users which have less than 5 ratings

In [7]:
userID_counts = ratings['userID'].value_counts()
ratings = ratings[ratings['userID'].isin(userID_counts[userID_counts >= 5].index)]

#### Merge ratings and videogames dataframe on productID and set to df dataframe

In [8]:
df = pd.merge(video_games,ratings, on="productID")
df.head(5)
df.shape

(6834, 12)

#### Create a pivot matrix with index userID, columns productID and values ratings

In [9]:
ratings_pivot = df.pivot(index='userID', columns='productID').ratings
ratings_pivot.head(5)

productID,9572132148,9861019731,9882155456,B000007VDN,B00000JBL1,B00000JDDR,B00000JDDY,B00000JDFT,B00000JKMB,B00000K11D,...,B00ILALMZS,B00IRHE892,B00ISW0TGW,B00IXHP3A4,B00IXHP4IK,B00IY46KR6,B00JF2FAP4,B00JF2FFR2,B00JM61264,B00KGGJPX6
userID,,,,,,,,,,,,,,,,,,,,,
A01803182IUSFNIFF5984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A05026511FQMVPVJH3KP8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A0607295133OZ2DIJW5P8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100JOT1GQ363B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A100O5QLORBZRO,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Set columns to productID variable

In [10]:
productID = ratings_pivot.columns
productID

Index([u'9572132148', u'9861019731', u'9882155456', u'B000007VDN',
       u'B00000JBL1', u'B00000JDDR', u'B00000JDDY', u'B00000JDFT',
       u'B00000JKMB', u'B00000K11D',
       ...
       u'B00ILALMZS', u'B00IRHE892', u'B00ISW0TGW', u'B00IXHP3A4',
       u'B00IXHP4IK', u'B00IY46KR6', u'B00JF2FAP4', u'B00JF2FFR2',
       u'B00JM61264', u'B00KGGJPX6'],
      dtype='object', name=u'productID', length=720)

#### Set index to userID variable

In [11]:
userID = ratings_pivot.index
userID

Index([u'A01803182IUSFNIFF5984', u'A05026511FQMVPVJH3KP8',
       u'A0607295133OZ2DIJW5P8', u'A100JOT1GQ363B', u'A100O5QLORBZRO',
       u'A102IYTHZ0FJ3I', u'A102MU6ZC9H1N6', u'A102R0714VSMB',
       u'A103B6MQ5IF2BK', u'A10566TEP2SO6I',
       ...
       u'AZQBBG4HIA0CQ', u'AZQHYOURLCLZH', u'AZQPD0AKFKYK1', u'AZTK5F3MOH16Q',
       u'AZUDF2ZSV6KK5', u'AZVCNOOQK36DH', u'AZYIAWJR9972L', u'AZYOVGJLQ03ML',
       u'AZZ1KF8RAO1BR', u'AZZNK89PXD006'],
      dtype='object', name=u'userID', length=4795)

In [12]:
correlated_items = ratings_pivot.corr()["B002I0JZOC"].sort_values(ascending=False).head(5)
correlated_items.index

Index([u'B002I0JZOC', u'B000LEFKXC', u'B0009VXBAQ', u'9572132148',
       u'9861019731'],
      dtype='object', name=u'productID')

#### Recommend video games with pearsonR

In [13]:
video_games[video_games["productID"].isin(correlated_items.index.tolist())]

,brand,categories,description,imUrl,price,productID,related,salesRank,title
1806410,None,"[[Video Games, Nintendo DS, Accessories, Cases...",Use the high quality silicone give your Ninten...,http://ecx.images-amazon.com/images/I/41%2BEXT...,NaN,9572132148,None,{u'Electronics': 176074},PINK Silicone Skin Case for Nintendo DS Lite /...
1817580,Generic,"[[Video Games, Wii, Accessories, Memory]]",Compatible with:\nNINTENDO Wii & Gamecube\nPac...,http://ecx.images-amazon.com/images/I/41pDBb4U...,4.42,9861019731,"{u'also_bought': [u'B002L8W5V6', u'B000ARPXU0'...",{u'Video Games': 579},64MB 1019 Block Memory Card compatible for Wii...
2485185,None,"[[Video Games, Wii]]",,http://ecx.images-amazon.com/images/I/41xM31Qi...,29.99,B0009VXBAQ,"{u'also_bought': [u'B000FQ9R4E', u'B002672Z80'...",{u'Video Games': 360},Wii
2894078,General,"[[Electronics, Computers & Accessories, Cables...",Data & Power USB Cable for Sony PSP,http://ecx.images-amazon.com/images/I/41q3EMWZ...,5.23,B000LEFKXC,"{u'also_bought': [u'B000ASBKHE', u'B007ZH9OFI'...",{u'Video Games': 2209},Data &amp; Power USB Cable for Sony PSP
4207281,Sony,"[[Video Games, PlayStation Vita, Games]]",,http://ecx.images-amazon.com/images/I/51DmnCeF...,399.99,B002I0JZOC,"{u'also_bought': [u'B006JKASCK', u'B0050SW8OS'...",{u'Video Games': 2721},Assassin's Creed III Liberation PlayStation Vi...


#### Recommend video games with cosine similarity

In [14]:
cosine_similarity_matrix = recommend_with_cosine_similarity(ratings_pivot)["B002I0JZOC"].sort_values(ascending=False).head(5)
cosine_similarity_matrix.index

Index([u'B002I0JZOC', u'B000LEFKXC', u'B0009VXBAQ', u'B00GOOSTJ0',
       u'B002MZZLNW'],
      dtype='object', name=u'productID')

#### Check if out recommendations in related column of dataset

In [15]:
video_games[video_games["productID"].isin(cosine_similarity_matrix.index.tolist())]

,brand,categories,description,imUrl,price,productID,related,salesRank,title
2485185,None,"[[Video Games, Wii]]",,http://ecx.images-amazon.com/images/I/41xM31Qi...,29.99,B0009VXBAQ,"{u'also_bought': [u'B000FQ9R4E', u'B002672Z80'...",{u'Video Games': 360},Wii
2894078,General,"[[Electronics, Computers & Accessories, Cables...",Data & Power USB Cable for Sony PSP,http://ecx.images-amazon.com/images/I/41q3EMWZ...,5.23,B000LEFKXC,"{u'also_bought': [u'B000ASBKHE', u'B007ZH9OFI'...",{u'Video Games': 2209},Data &amp; Power USB Cable for Sony PSP
4207281,Sony,"[[Video Games, PlayStation Vita, Games]]",,http://ecx.images-amazon.com/images/I/51DmnCeF...,399.99,B002I0JZOC,"{u'also_bought': [u'B006JKASCK', u'B0050SW8OS'...",{u'Video Games': 2721},Assassin's Creed III Liberation PlayStation Vi...
4273307,None,"[[Video Games, PC, Accessories, Gaming Mice]]",Infiltrate the world of state-of-the-art comba...,http://ecx.images-amazon.com/images/I/41xIiUOE...,39.99,B002MZZLNW,"{u'also_viewed': [u'B00C9R48A0', u'B00GX0ANIA'...",None,Call of Duty: Modern Warfare 2 Sniper Mouse
9027735,Atlus,"[[Video Games, PlayStation Vita, Games]]",Coming to Nintendo 3DS and PlayStation Vita is...,http://ecx.images-amazon.com/images/I/61UUMgyV...,36.86,B00GOOSTJ0,"{u'also_bought': [u'B00DVPMYXC', u'B00I3DOZ8O'...",{u'Video Games': 2016},Conception II: Children of the Seven Stars - P...
